In [ ]:
import torch

In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from torchvision import transforms
from tqdm import tqdm

In [ ]:
train_dataset=CIFAR100("./CIFAR100", download=True, transform=transforms.ToTensor())
test_dataset=CIFAR100("./CIFAR100", train=False, transform=transforms.ToTensor())

100%|██████████| 169M/169M [00:03<00:00, 47.0MB/s]


In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3, padding="same"), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
    nn.MaxPool2d((2, 2), stride=2),

    nn.Conv2d(64, 128, 3, padding="same"), nn.BatchNorm2d(128), nn.ReLU(inplace=True),

    nn.Conv2d(128, 256, 3, padding="same"), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
    nn.Dropout2d(),

    nn.Conv2d(256, 256, 3, padding="same"), nn.BatchNorm2d(256), nn.ReLU(inplace=True),

    nn.Conv2d(256, 128, 3, padding="same"), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
    nn.MaxPool2d((2, 2), stride=2),

    nn.Conv2d(128, 64, 3, padding="same"), nn.BatchNorm2d(64), nn.ReLU(inplace=True),

    nn.Flatten(),
    nn.Linear(4096, 256),
    nn.Dropout2d(),
    nn.ReLU(),
    nn.Linear(256, 100),
)

In [ ]:
device=torch.device("cuda")
model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Dropout2d(p=0.5, inplace=False)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (15): BatchNorm2d(128, eps=1e-05, momen

In [ ]:
g = torch.Generator(device='cuda')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, generator=g)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, generator=g)

lossfn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(100):
  model.train()
  running_correct = 0
  running_total = 0

  for batch in tqdm(train_dataloader):
      optimizer.zero_grad()
      x, y = batch
      x, y = x.to(device), y.to(device)
      outputs = model(x)
      loss = lossfn(outputs, y)

      l2_lambda = 0.001
      l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
      loss = loss + l2_lambda * l2_norm

      loss.backward()
      optimizer.step()

      # Accumulate training accuracy
      _, predicted = torch.max(outputs.data, 1)
      running_total += y.size(0)
      running_correct += (predicted == y).sum().item()

  train_accuracy = 100 * running_correct / running_total

  print(f"Epoch: {epoch}, train Loss: {loss.item()}, train accuracy: {train_accuracy:.2f}%")

  model.eval()
  running_correct = 0
  running_total = 0

  with torch.no_grad():
      for batch in tqdm(test_dataloader):
          x, y = batch
          x, y = x.to(device), y.to(device)
          outputs = model(x)
          loss = lossfn(outputs, y)

          _, predicted = torch.max(outputs.data, 1)
          running_total += y.size(0)
          running_correct += (predicted == y).sum().item()

  test_accuracy = 100 * running_correct / running_total
  print(f"Epoch: {epoch}, test Loss: {loss.item()}, test accuracy: {test_accuracy:.2f}%")



  0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
100%|██████████| 782/782 [00:23<00:00, 33.16it/s]


Epoch: 0, train Loss: 4.250352382659912, train accuracy: 19.89%


100%|██████████| 157/157 [00:02<00:00, 67.98it/s]


Epoch: 0, test Loss: 3.891636848449707, test accuracy: 28.27%


100%|██████████| 782/782 [00:23<00:00, 33.26it/s]


...

Epoch: 3, test Loss: 3.067741632461548, test accuracy: 31.27%


100%|██████████| 782/782 [00:23<00:00, 33.08it/s]


Epoch: 4, train Loss: 4.715912818908691, train accuracy: 24.32%


 38%|███▊      | 59/157 [00:01<00:01, 58.03it/s]


KeyboardInterrupt: 

In [ ]:
#   0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
#   warnings.warn(warn_msg)
# 100%|██████████| 782/782 [00:23<00:00, 32.94it/s]

# Epoch: 0, train Loss: 4.758344650268555, train accuracy: 10.99%

# 100%|██████████| 157/157 [00:02<00:00, 54.11it/s]

# Epoch: 0, test Loss: 3.5926952362060547, test accuracy: 23.94%

# 100%|██████████| 782/782 [00:23<00:00, 32.69it/s]

# Epoch: 1, train Loss: 4.341193199157715, train accuracy: 23.60%

# 100%|██████████| 157/157 [00:02<00:00, 66.27it/s]

# Epoch: 1, test Loss: 3.2581331729888916, test accuracy: 34.37%

# 100%|██████████| 782/782 [00:23<00:00, 33.12it/s]

# Epoch: 2, train Loss: 3.701869010925293, train accuracy: 31.73%

# 100%|██████████| 157/157 [00:02<00:00, 66.88it/s]

# Epoch: 2, test Loss: 2.703483819961548, test accuracy: 37.41%

# 100%|██████████| 782/782 [00:23<00:00, 33.14it/s]

# Epoch: 3, train Loss: 3.049783229827881, train accuracy: 37.47%

# 100%|██████████| 157/157 [00:02<00:00, 66.67it/s]

# Epoch: 3, test Loss: 2.194138288497925, test accuracy: 44.80%

# 100%|██████████| 782/782 [00:23<00:00, 32.77it/s]

# Epoch: 4, train Loss: 3.9603524208068848, train accuracy: 42.33%

# 100%|██████████| 157/157 [00:02<00:00, 64.89it/s]

# Epoch: 4, test Loss: 2.2496135234832764, test accuracy: 47.39%

# 100%|██████████| 782/782 [00:23<00:00, 32.98it/s]

# Epoch: 5, train Loss: 3.1045172214508057, train accuracy: 45.66%

# 100%|██████████| 157/157 [00:02<00:00, 66.79it/s]

# Epoch: 5, test Loss: 1.4738649129867554, test accuracy: 49.08%

# 100%|██████████| 782/782 [00:23<00:00, 32.97it/s]

# Epoch: 6, train Loss: 2.946915626525879, train accuracy: 48.92%

# 100%|██████████| 157/157 [00:02<00:00, 55.03it/s]

# Epoch: 6, test Loss: 1.9509754180908203, test accuracy: 50.07%

# 100%|██████████| 782/782 [00:23<00:00, 33.37it/s]

# Epoch: 7, train Loss: 2.4504878520965576, train accuracy: 51.62%

# 100%|██████████| 157/157 [00:02<00:00, 62.45it/s]

# Epoch: 7, test Loss: 2.3012735843658447, test accuracy: 50.19%

# 100%|██████████| 782/782 [00:23<00:00, 33.10it/s]

# Epoch: 8, train Loss: 2.379197597503662, train accuracy: 54.47%

# 100%|██████████| 157/157 [00:02<00:00, 67.65it/s]

# Epoch: 8, test Loss: 1.0628859996795654, test accuracy: 51.94%

# 100%|██████████| 782/782 [00:23<00:00, 33.25it/s]

# Epoch: 9, train Loss: 2.4079670906066895, train accuracy: 56.83%

# 100%|██████████| 157/157 [00:02<00:00, 67.19it/s]

# Epoch: 9, test Loss: 1.9797344207763672, test accuracy: 53.42%

# 100%|██████████| 782/782 [00:23<00:00, 33.06it/s]

# Epoch: 10, train Loss: 2.0972490310668945, train accuracy: 59.01%

# 100%|██████████| 157/157 [00:02<00:00, 67.71it/s]

# Epoch: 10, test Loss: 1.8001538515090942, test accuracy: 54.24%

# 100%|██████████| 782/782 [00:23<00:00, 32.91it/s]

# Epoch: 11, train Loss: 1.7727878093719482, train accuracy: 61.19%

# 100%|██████████| 157/157 [00:02<00:00, 66.85it/s]

# Epoch: 11, test Loss: 2.0373265743255615, test accuracy: 54.34%

# 100%|██████████| 782/782 [00:23<00:00, 32.95it/s]

# Epoch: 12, train Loss: 1.674553394317627, train accuracy: 63.52%

# 100%|██████████| 157/157 [00:02<00:00, 59.33it/s]

# Epoch: 12, test Loss: 1.6343984603881836, test accuracy: 53.32%

# 100%|██████████| 782/782 [00:23<00:00, 33.28it/s]

# Epoch: 13, train Loss: 2.1454408168792725, train accuracy: 65.49%

# 100%|██████████| 157/157 [00:02<00:00, 57.03it/s]

# Epoch: 13, test Loss: 1.635261058807373, test accuracy: 56.02%

# 100%|██████████| 782/782 [00:23<00:00, 32.97it/s]

# Epoch: 14, train Loss: 1.543928623199463, train accuracy: 67.55%

# 100%|██████████| 157/157 [00:02<00:00, 67.44it/s]

# Epoch: 14, test Loss: 2.2715306282043457, test accuracy: 55.30%

# 100%|██████████| 782/782 [00:23<00:00, 33.12it/s]

# Epoch: 15, train Loss: 2.1259913444519043, train accuracy: 69.81%

# 100%|██████████| 157/157 [00:02<00:00, 67.10it/s]

# Epoch: 15, test Loss: 2.810858726501465, test accuracy: 54.66%

# 100%|██████████| 782/782 [00:23<00:00, 33.15it/s]

# Epoch: 16, train Loss: 2.4269018173217773, train accuracy: 71.82%

# 100%|██████████| 157/157 [00:02<00:00, 66.56it/s]

# Epoch: 16, test Loss: 1.3604072332382202, test accuracy: 55.59%

# 100%|██████████| 782/782 [00:23<00:00, 33.16it/s]

# Epoch: 17, train Loss: 2.133638381958008, train accuracy: 73.36%

# 100%|██████████| 157/157 [00:02<00:00, 66.96it/s]

# Epoch: 17, test Loss: 1.3773679733276367, test accuracy: 55.00%

# 100%|██████████| 782/782 [00:23<00:00, 33.02it/s]

# Epoch: 18, train Loss: 1.6365013122558594, train accuracy: 75.35%

# 100%|██████████| 157/157 [00:02<00:00, 64.63it/s]

# Epoch: 18, test Loss: 1.736431360244751, test accuracy: 54.71%

# 100%|██████████| 782/782 [00:23<00:00, 33.10it/s]

# Epoch: 19, train Loss: 1.7056007385253906, train accuracy: 76.93%

# 100%|██████████| 157/157 [00:02<00:00, 53.85it/s]

# Epoch: 19, test Loss: 1.4214465618133545, test accuracy: 55.20%

# 100%|██████████| 782/782 [00:23<00:00, 33.00it/s]

# Epoch: 20, train Loss: 1.5306720733642578, train accuracy: 79.16%

# 100%|██████████| 157/157 [00:02<00:00, 66.84it/s]

# Epoch: 20, test Loss: 1.4956436157226562, test accuracy: 55.41%

# 100%|██████████| 782/782 [00:23<00:00, 33.04it/s]

# Epoch: 21, train Loss: 1.6582317352294922, train accuracy: 80.61%

# 100%|██████████| 157/157 [00:02<00:00, 66.28it/s]

# Epoch: 21, test Loss: 1.8348461389541626, test accuracy: 55.16%

#  30%|██▉       | 234/782 [00:07<00:16, 32.92it/s]
